In [2]:
import sys,os
import numpy
import heapq
import time
import json
import pandas as pd
import glob
from sklearn.feature_extraction import DictVectorizer

In [3]:
Train_All = False


In [4]:
training_files = glob.glob('data/training/*')
training_df = pd.DataFrame()
for index, filename in enumerate(training_files):
    with open(filename) as f:
        data = (line.strip() for line in f)
        data_json = "[{0}]".format(','.join(data))
    training_df = pd.concat([training_df, pd.read_json(data_json)], ignore_index=True)
    # for memory reserve
    if Train_All == False and index == 0: 
        break

In [5]:
training_df.head()

,author_id,browser,browser_version,category_id,city,cookie_pta,country,date,device_brand,device_marketing,hour,os,os_version,resolution,timestamp,url
0,naughtyangel,Other,,親子育兒,,74b87905e1b0cfce9bf065335ef95be39621636a7f27a5...,TW,20150412,,,15,Windows 7,,1347x1078,2015-04-12 07:42:07,http://naughtyangel.pixnet.net/blog/post/32214241
1,rita11836,Other,,美味食記,Taipei,1ee8750c0d7e36aea4471d94ee78030ba5cc7bcb123599...,TW,20150624,,,0,Windows 7,,1366x768,2015-06-23 16:11:33,http://rita11836.pixnet.net/blog/post/42034987
2,sezna627,Chrome,20a466bc8756f1f28988bf2e65fd9efeff055d6bb35ab3...,美味食記,,6f220bd289088782b0cf12665522f41ea7e616f09dd523...,TW,20150430,,,17,Windows XP,,1024x768,2015-04-30 09:32:05,http://sezna627.pixnet.net/blog/post/40824976
3,chifanlin,Chrome,4f5e86ee9a73052d24b5adf4811f2e17cdd12eaa57a39c...,不設分類,,c60ce22dc6d35bfb7fffb7f6fb37ce22ec7cfcd0b2063c...,TW,20150829,,,18,Windows 7,,1920x1200,2015-08-29 10:09:40,http://chifanlin.pixnet.net/blog/post/267531125
4,kthu1031,Mobile Safari,3e5ff76105726be5df3aa09262f402249a6edd04bd8ebb...,食譜分享,,64276935bf38bad0f00b98265e4a861b2bef0b6d9935b8...,TW,20150605,Apple,c78389efe59a903a04382eab0da9bc0412d288433897f6...,20,iOS,2bb85d70ca4ffbc3b75cd1bf1a3c66f2260571fb0ccba1...,768x1024,2015-06-05 12:57:41,http://kthu1031.pixnet.net/blog/post/6370860


In [127]:
# encode str col
trainEnc_df = training_df.copy()
encode_dict = {}

M_dict = [ {} for _ in range(11)]
def encode_series(series):
    x = series.astype('category')
    encode_series = x.cat.codes
    dict_df = pd.concat([x.cat.codes, series], axis=1)
    series_dict = dict_df.drop_duplicates().sort_values(0).set_index(0).to_dict()
    
    return encode_series, series_dict
trainEnc_df.browser           ,M_dict[0] = encode_series(trainEnc_df.browser); 
trainEnc_df.browser_version   ,M_dict[1] = encode_series(trainEnc_df.browser_version)
trainEnc_df.category_id       ,M_dict[2] = encode_series(trainEnc_df.category_id)
trainEnc_df.city              ,M_dict[3] = encode_series(trainEnc_df.city)
trainEnc_df.country           ,M_dict[4] = encode_series(trainEnc_df.country)
trainEnc_df.device_brand      ,M_dict[5] = encode_series(trainEnc_df.device_brand)
trainEnc_df.device_marketing  ,M_dict[6] = encode_series(trainEnc_df.device_marketing)
trainEnc_df.os                ,M_dict[7] = encode_series(trainEnc_df.os)
trainEnc_df.os_version        ,M_dict[8] = encode_series(trainEnc_df.os_version)
trainEnc_df.resolution        ,M_dict[9] = encode_series(trainEnc_df.resolution)
trainEnc_df.cookie_pta        ,M_dict[10]= encode_series(trainEnc_df.cookie_pta)

for i in range(11):
    encode_dict.update(M_dict[i])



In [128]:
type(trainEnc_df.cookie_pta[0])
print(trainEnc_df.cookie_pta.shape, trainEnc_df.browser.shape)

(462677,) (462677,)


In [131]:
from sklearn.svm import SVC
def M_findDeviceByCookie(browser, cookie):
    clf = SVC(gamma='auto')
    score = clf.fit(browser, cookie)
    return score, clf
score, M_clf = M_findDeviceByCookie(trainEnc_df.browser, trainEnc_df.cookie_pta)
print(score)

ValueError: Expected 2D array, got 1D array instead:
array=[19. 19.  3. ...  0.  0. 15.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [29]:
class PIXNET_Recommendation:
    
    def __init__(self,training_data_folder="data/training",test_data_file_path="data/testing.json",target_file_path="tmp/recommendation",k=20):
        self.training_data_folder = training_data_folder
        self.test_data_file_path = test_data_file_path
        self.target_file_path = target_file_path
        self.k = k
    
    def load_data(self):
        data_set ={}
        training_files = os.listdir(self.training_data_folder)
        print(training_files)
        for filename in training_files:
            for line in open("%s/%s"%(self.training_data_folder,filename)):
                data_json = json.loads(line.strip())
                if data_json['cookie_pta'] not in data_set:
                    data_set[data_json['cookie_pta']]={}
                data_set[data_json['cookie_pta']][data_json['author_id']] = 1
        v = DictVectorizer(sparse=True)
        X = v.fit_transform(data_set.values()).T
        author = v.get_feature_names()
        self.data_set =data_set
        self.X = X
        self.author = author
    
    def calculate_item_similarity(self):
        X = self.X
        author = self.author
        distance = sum(X.T.toarray())
        similarity_tmp_file = open("tmp/similarity_top10","w")
        count = 0
        size_per_loop =1000
        while count< X.shape[0]:
            size_max = min(count+size_per_loop,X.shape[0])-count
            similarity = X[count:min(count+size_per_loop,X.shape[0])].dot(X.T).toarray()
            for i in xrange(size_max):
                similarity[i] = similarity[i]/numpy.sqrt(distance[count+i])
            similarity = similarity.T
            for i in xrange(X.shape[0]):
                similarity[i] = numpy.round(similarity[i]/numpy.sqrt(distance[i]),2)
            similarity = similarity.T
            for i in xrange(size_max):
                rank = []
                for j in xrange(X.shape[0]):
                    if similarity[i][j] >0.99:
                        similarity[i][j] = 0
                    rank.append(similarity[i][j])
                top_k_similarity = heapq.nlargest(self.k, xrange(len(rank)), rank.__getitem__)
                data =[]
                for j in xrange(len(top_k_similarity)):
                    data.append("%s,%.2f"%(author[top_k_similarity[j]],rank[top_k_similarity[j]]))
                similarity_tmp_file.write(";".join(data)+"\n")
            del similarity
            count+=size_per_loop
        similarity_tmp_file.close()
    
    def load_recommend_target(self):
        target_file = open(self.test_data_file_path)
        questions = {}
        for i in target_file:
            j =json.loads(i)
            questions[j['cookie_pta']] = 1
        target_file.close()
        self.questions= questions
    
    def recommend_from_item_similarity(self):
        data_set =self.data_set
        questions = self.questions
        author = self.author
        f = open("tmp/similarity_top10")
        count =0
        author_recommend = {}
        for i in f:
            r =i.strip().split(";")
            author_recommend[author[count]] = {}
            for j in r:
                a = j.split(",")
                author_recommend[author[count]][a[0]] = float(a[1])
            count +=1
        f_out = open(self.target_file_path,"w")
        for i in questions.keys():
            log = data_set[i]
            recommend_author = {}
            for author_id in log.keys():
                for r_author in author_recommend[author_id]:
                    if r_author in log:
                        continue
                    if r_author not in recommend_author:
                        recommend_author[r_author] = 0
                    recommend_author[r_author] += author_recommend[author_id][r_author]
            js = {"cookie_pta":i,"author_id":heapq.nlargest(len(recommend_author)/20,recommend_author,key=recommend_author.__getitem__)}
            f_out.write(json.dumps(js)+"\n")
        f_out.close()

    def main(self):
        self.load_data()
        self.calculate_item_similarity()
        self.load_recommend_target()
        self.recommend_from_item_similarity()

In [14]:
recommendation = PIXNET_Recommendation()
recommendation.main()

['part-00022', 'part-00016', 'part-00012', 'part-00020', 'part-00001', 'part-00005', 'part-00006', 'part-00023', 'part-00008', 'part-00015', 'part-00007', 'part-00002', 'part-00003', 'part-00013', 'part-00000', 'part-00017', 'part-00010', 'part-00014', 'part-00004', 'part-00018', 'part-00011', 'part-00021', 'part-00009', 'part-00019']


KeyboardInterrupt: 